# PACKAGES

In [1]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
%matplotlib inline
import matplotlib.pyplot as plt

# DONNEES

In [2]:
print('total training damage images: ',len(os.listdir("./train_another/damage")))
print('total validation damage images: ',len(os.listdir("./validation_another/damage")))
print('total test damage images: ',len(os.listdir("./test_another/damage")))
print('total training no damage images: ',len(os.listdir("./train_another/no_damage")))
print('total validation no damage images: ',len(os.listdir("./validation_another/no_damage")))
print('total test no damage images: ',len(os.listdir("./test_another/no_damage")))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  8000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [3]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])
batch_size = 5000

df_train = datasets.ImageFolder("./train_another",transform=transform)
train_dataloader = DataLoader(dataset=df_train, batch_size=batch_size, shuffle=True)

df_test = datasets.ImageFolder("./test_another",transform=transform)
test_dataloader = DataLoader(dataset=df_test, batch_size=batch_size, shuffle=True)

df_validation = datasets.ImageFolder("./validation_another",transform=transform)
test_dataloader = DataLoader(dataset=df_validation, batch_size=batch_size, shuffle=True)

# MODELISATION

## VGG-16

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self


In [4]:
import torchvision

model = torchvision.models.vgg16(pretrained=True)
print(model)

c:\Users\spica\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\spica\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Paramètre de l'apprentissage

In [5]:
loss_fn = nn.BCELoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
num_epochs = 5

NameError: name 'nn' is not defined

## Rétropagation

In [10]:
loss_values = []

for epoch in range(num_epochs):
    for X, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()

print("Training Complete")

KeyboardInterrupt: 

## Méthode chatgpt

Notez que le modèle VGG-16 pré-entraîné a déjà été formé sur une grande quantité de données et que ses poids ont déjà été optimisés. Il est généralement conseillé de le réutiliser comme bloc de base pour une tâche de reconnaissance d'images spécifique et de l'ajuster en entraînant seulement la dernière couche complètement connectée, plutôt que de le réentraîner à partir de zéro.

In [6]:
num_classes = 1000

# Verrouillage des couches du modèle (sauf la dernière) pour éviter leur mise à jour lors de l'entraînement
for param in model.parameters():
    param.requires_grad = False

# Modification du nombre de classes en sortie (par défaut 1000) si nécessaire
model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=num_classes)

# Déverrouillage de la dernière couche pour permettre son entraînement
for param in model.classifier[6].parameters():
    param.requires_grad = True

# Conversion en mode "entraînement"
model.train()

# Préparation des transformateurs pour les données d'entraînement
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

batch_size = 10
num_epochs = 10

trainset = torchvision.datasets.ImageFolder(root='./train_another', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
loss_values = []

# Boucle d'entraînement
for epoch in range(num_epochs):
    for i, data in enumerate(trainloader, 0):
        # Chargement des entrées et des étiquettes
        inputs, labels = data
        
        # Calcul des sorties à l'aide de la méthode forward
        outputs = model(inputs)
        
        # Calcul de la perte
        criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(outputs, labels)
        
        # Ajout de la perte à la liste
        loss_values.append(loss.item())
        # Calcul des gradients pour la rétropropagation
        optimizer = torch.optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)
        optimizer.zero_grad()
        loss.backward()
        
        # Mise à jour des paramètres du modèle
        optimizer.step()
        
        # Affichage de la progression de l'entraînement toutes les 100 itérations
        if i ==10:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss.item()))
            break

[1,    11] loss: 5.317
[2,    11] loss: 3.123
[3,    11] loss: 2.764
[4,    11] loss: 1.549
[5,    11] loss: 0.805
[6,    11] loss: 0.670
[7,    11] loss: 1.274
[8,    11] loss: 0.622
[9,    11] loss: 0.593
[10,    11] loss: 0.711


In [2]:
# Tracé de la perte au cours de l'entraînement
plt.plot(test)
plt.xlabel('Itération')
plt.ylabel('Perte')
plt.show()

: 

: 